# Setting Up the Database


*   duckdb is the database we use here
*   dbt-duckdb is a tool that is required to run dbt models on local DuckDB files

In [ ]:
# Install dbt and DuckDB
%pip install dbt-duckdb duckdb

In [ ]:
# Create main project folder
!mkdir -p /content/my_dbt_project/models

In [ ]:
# Move into it
%cd /content/my_dbt_project

/content/my_dbt_project


Set up **dbt Configuration** telling how to connect to database


*   *profiles.yml* contains all the connection details



In [ ]:
profile_content = """
my_dbt_project:
  target: dev
  outputs:
    dev:
      type: duckdb
      path: /content/my_dbt_project/my_duckdb.duckdb
"""

# Create the .dbt directory in the root
!mkdir -p /root/.dbt

with open("/root/.dbt/profiles.yml", "w") as f:
    f.write(profile_content)

Setting up the **main configuration** file for your dbt project, called *dbt_project.yml*

In [ ]:
dbt_project_content = """
name: 'my_dbt_project'
version: '1.0'
config-version: 2

profile: 'my_dbt_project'

model-paths: ["models"]
target-path: "target"
clean-targets:
  - "target"
  - "dbt_modules"
"""

with open("/content/my_dbt_project/dbt_project.yml", "w") as f:
    f.write(dbt_project_content)

📁 Create the models/ folder and write a model (order_summary.sql)

🧾 Create the CSV file (orders.csv)

🛠️ Load the CSV into DuckDB (raw_orders)

▶️ Run dbt run to build the model

### 1️⃣ Upload CSV Files to Google Colab


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving orders.csv to orders (1).csv
Saving products.csv to products (1).csv


### 2️⃣ Load CSV Files into DuckDB `

In [ ]:
import duckdb

# Connect to DuckDB
con = duckdb.connect("/content/my_dbt_project/my_duckdb.duckdb")

# Create tables from CSV files
con.execute("CREATE OR REPLACE TABLE raw_orders AS SELECT * FROM read_csv_auto('/content/my_dbt_project/orders.csv')")
con.execute("CREATE OR REPLACE TABLE raw_products AS SELECT * FROM read_csv_auto('/content/my_dbt_project/products.csv')")

print("Tables loaded into DuckDB")

Tables loaded into DuckDB


This will create two tables in DuckDB:

*   raw_orders (from orders.csv)
*   raw_products (from products.csv)

### 3️⃣ Create Your First dbt Model

In [ ]:
model_sql = """
SELECT
    o.order_id,
    o.customer_id,
    o.order_date,
    o.quantity,
    o.unit_price AS order_unit_price,
    p.product_name,
    p.category,
    p.unit_price AS product_unit_price,
    (o.quantity * o.unit_price) AS total_order_price
FROM
    raw_orders o
JOIN
    raw_products p
ON
    o.product_id = p.product_id
"""

with open("/content/my_dbt_project/models/orders_with_products.sql", "w") as f:
    f.write(model_sql)


In [ ]:
con.close()

Executing dbt Models with dbt run

In [ ]:
%cd /content/my_dbt_project
!dbt run

/content/my_dbt_project
14:15:08  Running with dbt=1.9.4
14:15:09  Registered adapter: duckdb=1.9.3
14:15:09  Found 1 model, 428 macros
14:15:09  
14:15:09  Concurrency: 1 threads (target='dev')
14:15:09  
14:15:09  1 of 1 START sql view model main.orders_with_products .......................... [RUN]
14:15:10  1 of 1 OK created sql view model main.orders_with_products ..................... [OK in 0.14s]
14:15:10  
14:15:10  Finished running 1 view model in 0 hours 0 minutes and 0.32 seconds (0.32s).
14:15:10  
14:15:10  Completed successfully
14:15:10  
14:15:10  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1


Query dbt-Created View Using DuckDB SQL in Python

In [ ]:
# Reconnect to DuckDB if needed
con = duckdb.connect("/content/my_dbt_project/my_duckdb.duckdb")

# Query the new model output
result = con.execute("SELECT * FROM orders_with_products").fetchall()
print(result)

[(1, 'C001', datetime.date(2023, 8, 1), 2, 19.99, 'Wireless Mouse', 'Electronics', 19.99, 39.98), (2, 'C002', datetime.date(2023, 8, 3), 1, 149.99, 'Gaming Headset', 'Electronics', 149.99, 149.99), (3, 'C001', datetime.date(2023, 8, 4), 3, 9.99, 'Keyboard', 'Electronics', 9.99, 29.97), (4, 'C003', datetime.date(2023, 8, 5), 5, 19.99, 'Wireless Mouse', 'Electronics', 19.99, 99.94999999999999), (5, 'C002', datetime.date(2023, 8, 6), 2, 9.99, 'Keyboard', 'Electronics', 9.99, 19.98), (6, 'C004', datetime.date(2023, 8, 6), 1, 249.99, '4K Monitor', 'Electronics', 249.99, 249.99)]


Fetch dbt-Transformed Data as a DataFrame Using DuckDB and Pandas

In [ ]:
import duckdb
import pandas as pd

# Reconnect to the DuckDB database
con = duckdb.connect("/content/my_dbt_project/my_duckdb.duckdb")

# Run the query and load the results into a DataFrame
df = con.execute("SELECT * FROM orders_with_products").fetchdf()

# Show the table
df.head()  # Show first 5 rows (or just use df to show all)

,order_id,customer_id,order_date,quantity,order_unit_price,product_name,category,product_unit_price,total_order_price
0,1,C001,2023-08-01,2,19.99,Wireless Mouse,Electronics,19.99,39.98
1,2,C002,2023-08-03,1,149.99,Gaming Headset,Electronics,149.99,149.99
2,3,C001,2023-08-04,3,9.99,Keyboard,Electronics,9.99,29.97
3,4,C003,2023-08-05,5,19.99,Wireless Mouse,Electronics,19.99,99.95
4,5,C002,2023-08-06,2,9.99,Keyboard,Electronics,9.99,19.98
